In [ ]:
!pip -q install openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 855.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.7 MB/s eta 0:00:00


In [ ]:
import os

In [ ]:
#Setting the key
openai_api_key='your key'
os.environ['OPENAI_API_KEY'] = openai_api_key

# Question Generation Function

In [ ]:
#Input num_of_ques any value if you want to generate questions such that it covers all the text

#If a particular number of questions are desired then uncomment the code in the function

def generate_ques(llm_model,text,num_of_ques):
  from langchain.chat_models import ChatOpenAI
  from langchain.schema import HumanMessage, SystemMessage, AIMessage
  from langchain.llms import OpenAI
  from langchain.chat_models import ChatOpenAI
  from langchain import PromptTemplate, LLMChain
  from langchain.prompts.chat import(
      ChatPromptTemplate,
      SystemMessagePromptTemplate,
      AIMessagePromptTemplate,
      HumanMessagePromptTemplate
  )

  from langchain.schema import(
      AIMessage,
      HumanMessage,
      SystemMessage
  )
  chat=ChatOpenAI(temperature=0.5)
  messages=[
      #Uncomment the command below to generate a particular number of questions
      #SystemMessage(content="Generate {0} of questions only from the text given by user".format(num_of_ques)),
    SystemMessage(content="Generate a particular number of questions from the text given by user such that the questions covers all the information in the text"),
    HumanMessage(content=text)
  ]
  resp= chat(messages)
  resl=resp.content
  return resl



# Batch Division of questions

In [ ]:
def divide_batches(ques, max_questions_per_batch):
  import re
  input_string=ques
  #Splitting questions into a single element of a list
  questions=re.split(r'\n',input_string)

  import math
  # Grouping them in bacthes of desired size
  num_batches = math.ceil(len(questions) / max_questions_per_batch)
  batches = []

  for i in range(num_batches):
      start_idx = i * max_questions_per_batch
      end_idx = (i + 1) * max_questions_per_batch
      batch = questions[start_idx:end_idx]
      batches.append(batch)

  return batches



"  import re\n  input_string=questions\n  #Splitting questions into a single element of a list\n  ques=re.split(r'\n',input_string)"

# Answer Generation Function

In [ ]:
def answer(ques,context):
  from langchain.chat_models import ChatOpenAI
  from langchain.schema import HumanMessage, SystemMessage, AIMessage
  from langchain.llms import OpenAI
  from langchain.chat_models import ChatOpenAI
  from langchain import PromptTemplate, LLMChain
  from langchain.prompts.chat import(
      ChatPromptTemplate,
      SystemMessagePromptTemplate,
      AIMessagePromptTemplate,
      HumanMessagePromptTemplate
  )

  from langchain.schema import(
      AIMessage,
      HumanMessage,
      SystemMessage
  )
  chat=ChatOpenAI(temperature=0.5)
  ans=[]
  n=len(ques)
  for i in range(n):
    for q in ques[i]:
      messages=[
      SystemMessage(content="Answer within the scope of {0}".format(context)),
      HumanMessage(content=q)
      ]
      resp= chat(messages)
      ans.append(resp.content)

  return ans



# Storing in JSON Format

In [ ]:
# A function to store the output in json format, it will give a txt file on being runned
# Output file name , questions and answers list are passed as parameters


def json_write_to_file(file_name, ques,ans):
  n=len(ques)
  l=[]
  for i in range(n):
    for x in ques[i]:
      l.append(x)
  file_name=file_name
  import json
  with open(file_name,'w') as file:
    for x,y in zip(l,ans):
      data={"prompt":x,"completion":y}
      formatted_data=json.dumps(data)
      file.write(formatted_data)
      file.write('\n')





# Illustrating with an example

In [ ]:
#Taking an article for example

article='Fuel is any substance used to produce heat and energy through a chemical or nuclear reaction. The energy is used by the conversion of a portion of the fuels mass. In India, we are suffering a severe fuel crises. In view of this, the Petroleum Conservation Research Association is encouraging fuel conservation. Our goal should be to reduce gasoline consumption progressively each year.'

In [ ]:
# Training a llm
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.llms import OpenAI
chat = ChatOpenAI(temperature=.7, openai_api_key=openai_api_key)

llm = OpenAI(model_name='text-davinci-003',
             temperature=0.9,
             max_tokens = 256)


from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import(
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)

from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)
chat=ChatOpenAI(temperature=0.5)

### Generating questions

In [ ]:
# Generating questions such that all text is covered
ques= generate_ques(chat, article, 5)

In [ ]:
ex1=ques
ex1

"1. What is fuel and how is it used to produce heat and energy?\n2. What is the role of the Petroleum Conservation Research Association in India?\n3. Why is there a fuel crisis in India?\n4. How does the conversion of a portion of the fuel's mass contribute to the production of energy?\n5. What is the goal of the Petroleum Conservation Research Association in terms of gasoline consumption?\n6. How does the association encourage fuel conservation?\n7. What are the potential consequences of not reducing gasoline consumption in India?\n8. Are there any alternative sources of energy being promoted by the association?\n9. What are some measures individuals can take to conserve fuel?\n10. How can reducing gasoline consumption help alleviate the fuel crisis in India?"

### Dividing into batches

In [ ]:
#Diving into batches of 4 each
ques=divide_batches(ex1,4)

In [ ]:
ques

[['1. What is fuel and how is it used to produce heat and energy?',
  '2. What is the role of the Petroleum Conservation Research Association in India?',
  '3. Why is there a fuel crisis in India?',
  "4. How does the conversion of a portion of the fuel's mass contribute to the production of energy?"],
 ['5. What is the goal of the Petroleum Conservation Research Association in terms of gasoline consumption?',
  '6. How does the association encourage fuel conservation?',
  '7. What are the potential consequences of not reducing gasoline consumption in India?',
  '8. Are there any alternative sources of energy being promoted by the association?'],
 ['9. What are some measures individuals can take to conserve fuel?',
  '10. How can reducing gasoline consumption help alleviate the fuel crisis in India?']]

In [ ]:
n=len(ques)

### Answering the questions

In [ ]:
ans=answer(ques,article)

In [ ]:
ans

### Storing in json

In [ ]:
file_name = "out.txt"


json_write_to_file(file_name, ques,ans)

print("Output has been written to", file_name)

Output has been written to out.txt


In [ ]:
#We get a file named out.txt and this can be directly used for fine-tuning